In [ ]:
#!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

In [ ]:
!pip3 install --no-cache-dir -U crcmod

In [ ]:
!gsutil cp gs://paper-2-data/archive.zip .

Copying gs://paper-2-data/archive.zip...
/ [1 files][  3.8 GiB/  3.8 GiB]   70.2 MiB/s                                   
Operation completed over 1 objects/3.8 GiB.                                      


In [ ]:
import zipfile
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf
import keras
import dask.dataframe as dd
import os

In [ ]:
np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=np.inf)

In [ ]:
df_train = dd.read_csv('train.csv', dtype={'id': 'object'})
df_val = dd.read_csv('valid.csv', dtype={'id': 'object'})

In [ ]:
labels = {'real':1, 'fake':0}

In [ ]:
def path_to_pixels(image_path):
    path = '/content/real_vs_fake/real-vs-fake/'
    im = Image.open(path+image_path)
    pixels = np.array(im.getdata()).flatten()
    return pixels

In [ ]:
num_par = 15
df_test = dd.read_csv('test.csv', dtype={'id': 'object'})
df_test = df_test.repartition(npartitions=num_par)
output_path='test_data.csv'
for i in range(num_par):
  if i%2==0:
    print(i)
  df_test1 = df_test.get_partition(i)
  df_test1['pixels'] = df_test1['path'].apply(path_to_pixels, meta=('path_to_pixels', object))
  df_test1.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
!gsutil cp test_data.csv gs://paper-2-data/

0
2
4
6
8
10
12
14
Copying file://test_data.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][ 14.6 GiB/ 14.6 GiB]   78.1 MiB/s                                   
Operation completed over 1 objects/14.6 GiB.                                     
